In [0]:
# 改成这个版本的sklearn才不会报错
!pip install scikit-learn==0.21.2

     |████████████████████████████████| 6.7MB 2.8MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [0]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import pickle

In [0]:
# 当前的路径
import os
os.chdir("/content/drive/My Drive/recommention")

In [0]:
train = pickle.load(open('trainset_plus.pickle', 'rb'))
train_label = pickle.load(open('trainset_label.pickle', 'rb'))

In [0]:
from numpy import random

In [0]:
train_label[3]

90

In [0]:
# 将数据集打乱
random.seed(12)
random.shuffle(train)
random.seed(12)
random.shuffle(train_label)

In [0]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
# one hot 编码
one_hot_train_label = to_categorical(train_label)

In [0]:
one_hot_train_label.shape

(4008915, 101)

In [0]:
data_feature = train.shape[1]
data_feature

9

In [0]:
# 定义神经网络
from keras import models
from keras import layers
from keras import optimizers
from keras import metrics
import keras

In [0]:
# 选出10000个作为验证集
x_val = train[:10000]
partial_x_train = train[10000:]

y_val = one_hot_train_label[:10000]
partial_y_train = one_hot_train_label[10000:]

In [0]:
#实现随机超参数搜索

#1.将tf.keras.models。sequential转化为sklearn的model

#封装模型
def bulid_model(hidden_layers =1, layer_size = 30, learning_rate = 3e-3):
  model = models.Sequential()
  model.add(layers.Dense(layer_size, activation='relu', input_shape=(data_feature,)))
   
  for _ in range(hidden_layers -1):
    model.add(layers.Dense(layer_size, activation="relu"))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(101, activation='softmax'))
    #定义优化函数，使用自定义的learning_rate
  rmsprop = optimizers.RMSprop(lr=learning_rate, rho=0.9, epsilon=1e-06)
  model.compile(optimizer=rmsprop,
              loss='mse',
              # metrics=[metrics.mae, metrics.categorical_accuracy]
              metrics=['acc']
              )
  return model

In [0]:
# 转化模型  tf.keras.wrappers.scikit_learn.KerasClassifier
sklearn_model = keras.wrappers.scikit_learn.KerasClassifier(bulid_model)

In [0]:
#2.定义要调整的超参数集合
from  scipy.stats import reciprocal
# f(x) = 1/(x*log(b/a))   a <=  x <= b  

param_distribution = {
    "hidden_layers": [2, 3, 4,5, 6],
    "layer_size": np.arange(1, 200),
    "learning_rate":reciprocal(1e-4, 1e-2)
}


In [0]:
# 3.参数搜索 RandomizedSearchCV

from sklearn.model_selection import RandomizedSearchCV
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
# RandomizedSearchCV参数说明，
# clf1设置训练的学习器
# param_dist字典类型，放入参数搜索范围
# scoring = 'neg_log_loss'，精度评价方式设定为"neg_log_loss"
# n_iter=300，训练300次，数值越大，获得的参数精度越大，但是搜索时间越长
# n_jobs = -1，使用所有的CPU进行训练，默认为1，使用1个CPU
# RandomizedSearchCV采用了cross-validation: 将训练集分成n分，n-1训练，最后一份验证。默认cv=3
random_search_cv = RandomizedSearchCV(sklearn_model,
                                      param_distribution,
                                      cv = 3,
                                      n_iter = 20,
                                      n_jobs = -1)
# use call back function
callbacks = [
    # 目标指标不再有改善了，就可以提前终止
    # 
    EarlyStopping(monitor='val_acc', # 被检测的模型的指标
                  patience=2, # 没有进步的训练轮次为2，在这之后训练就会被停止
                  # min_delta=1e-3
                  verbose=1),# 展示信息
    # 保存模型 
    ModelCheckpoint(filepath="model_{epoch:03d}-{val_acc:.4f}.h5",# 路径
                                    monitor='val_acc',# 如果val_loss没有改善就不覆盖
                                    save_best_only=True)# 保存最佳模型

]

In [0]:
# train model
random_search_cv.fit(partial_x_train,
                     partial_y_train,
                     epochs = 30, 
                     validation_data=(x_val, y_val), 
                     batch_size=512,
                     callbacks=callbacks
                    )
print(random_search_cv.best_params_)
print(random_search_cv.best_score_)
print(random_search_cv.best_estimator_)
# choose the best model 
model = random_search_cv.best_estimator_.model
model.save('net_model_v1.h5')

test = pd.read_csv('testset_plus.csv')
# 获取label
test_label = test['score'].tolist()
# 删除score这一列
test.drop('score', axis=1, inplace=True)
# 归一化
test = test.apply(lambda x : (x-np.min(x))/(np.max(x)-np.min(x)))
test = test.values
# one hot 编码
one_hot_test_label = to_categorical(test_label)
# 评估模型
VERBOSE = 1
score = model.evaluate(x=test,y=one_hot_test_label, verbose=VERBOSE)
print('Test score:', score[0])
print('Test accuracy:', score[1])
from keras.applications.imagenet_utils import decode_predictions
preds = model.predict(test)
# 解热码
decoded_data = []
for i in range(preds.shape[0]):
      decoded = np.argmax(preds[i], axis=0)
      decoded_data.append(decoded)
decoded_data = np.array(decoded_data)
from sklearn.metrics import mean_squared_error
# 计算rmse
rmse = np.sqrt(mean_squared_error(test_label,decoded_data))
rmse
# model.evaluate(x_test_scaled, y_test)
# save model
# model.save('net_model_v1.h5')

KeyboardInterrupt: ignored